In [ ]:
import requests

# replace the "demo" apikey below with your own key from https://www.alphavantage.co/support/#api-key
# url = 'https://www.alphavantage.co/query?function=CPI&interval=monthly&apikey='
url = 'https://www.alphavantage.co/query?function=INFLATION&interval=monthly&apikey='
r = requests.get(url)
data = r.json()

print(data)

In [ ]:
https://api.worldbank.org/v2/country/vn/indicator/SP.POP.TOTL?date=2020&format=json

In [1]:
base_inflation_url = 'https://api.worldbank.org/v2/country/US/indicator/FP.CPI.TOTL.ZG?date=1950:2023&format=json&page=2'
COUNTRY_CODE = 'USA'
START_YEAR = 1950
END_YEAR = 2025
page = 1

In [2]:
full_url = f'https://api.worldbank.org/v2/country/{COUNTRY_CODE}/indicator/SP.POP.TOTL?date={START_YEAR}:{END_YEAR}&format=json'
import asyncio
import aiohttp
import pandas as pd
import json

async def fetch_data(session, url):
    async with session.get(url) as response:
        return await response.json()

async def main():
    async with aiohttp.ClientSession() as session:
        data = await fetch_data(session, full_url)
        # df = pd.json_normalize(data)
        # print(df)
        return data

In [3]:
import requests
response = requests.get(full_url)

In [ ]:
type(response.content)

In [ ]:
page = response.json()

In [ ]:
page[1]

In [4]:
import requests

url = "https://api.worldbank.org/v2/country/USA/indicator/FP.CPI.TOTL.ZG?date=1950:2025&format=json"
r = requests.get(url)
data = r.json()[1]  # phần dữ liệu


In [7]:
data_1 = r.json()[0]  # phần metadata
data_2 = r.json()[1]  # phần dữ liệu

In [11]:
from pandas import json_normalize
df = json_normalize(data_2)
df.head()

,countryiso3code,date,value,unit,obs_status,decimal,indicator.id,indicator.value,country.id,country.value
0,USA,2024,2.949525,,,1,FP.CPI.TOTL.ZG,"Inflation, consumer prices (annual %)",US,United States
1,USA,2023,4.116338,,,1,FP.CPI.TOTL.ZG,"Inflation, consumer prices (annual %)",US,United States
2,USA,2022,8.002800,,,1,FP.CPI.TOTL.ZG,"Inflation, consumer prices (annual %)",US,United States
3,USA,2021,4.697859,,,1,FP.CPI.TOTL.ZG,"Inflation, consumer prices (annual %)",US,United States
4,USA,2020,1.233584,,,1,FP.CPI.TOTL.ZG,"Inflation, consumer prices (annual %)",US,United States


In [13]:
all_dfs = []  # danh sách chứa các DataFrame của từng trang

page_range = data_1['pages']
for i in range(1, page_range + 1):
    paged_url = url + f'&page={i}'
    r = requests.get(paged_url)
    data = r.json()[1]
    df = json_normalize(data)
    all_dfs.append(df)  # Thêm DataFrame vào danh sách

final_df = pd.concat(all_dfs, ignore_index=True)

In [17]:
print(final_df.shape)
print(final_df.info())

(65, 10)
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 65 entries, 0 to 64
Data columns (total 10 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   countryiso3code  65 non-null     object 
 1   date             65 non-null     object 
 2   value            65 non-null     float64
 3   unit             65 non-null     object 
 4   obs_status       65 non-null     object 
 5   decimal          65 non-null     int64  
 6   indicator.id     65 non-null     object 
 7   indicator.value  65 non-null     object 
 8   country.id       65 non-null     object 
 9   country.value    65 non-null     object 
dtypes: float64(1), int64(1), object(8)
memory usage: 5.2+ KB
None


In [22]:
country_iso_code = pd.read_csv('/home/anhtt1/Workspace/DE/Project/DE_Project/projects/AlphaVantage/data/country_codes.csv')
country_iso_code.head()

,country,alpha_2_code,alpha_3_code,numeric
0,Afghanistan,AF,AFG,4
1,Åland Islands,AX,ALA,248
2,Albania,AL,ALB,8
3,Algeria,DZ,DZA,12
4,American Samoa,AS,ASM,16


In [26]:
for i in range(country_iso_code.shape[0]):
    final_df.loc[final_df['countryiso3code'] == country_iso_code.loc[i, 'alpha_3_code'], 'country_name'] = country_iso_code.loc[i, 'country']

In [29]:
import requests
import pandas as pd

START_YEAR = 1950
END_YEAR = 2025

for idx in range(country_iso_code.shape[0]):
    alpha3 = country_iso_code.loc[idx, 'alpha_3_code']
    full_url = f"https://api.worldbank.org/v2/country/{alpha3}/indicator/SP.POP.TOTL?date={START_YEAR}:{END_YEAR}&format=json"

    r = requests.get(full_url)
    json_data = r.json()

    # Kiểm tra dữ liệu hợp lệ
    if not json_data or len(json_data) < 2 or json_data[1] is None:
        print(f"⚠ Không có dữ liệu cho {alpha3}")
        continue

    data_1 = json_data[0]  # metadata
    all_dfs = []

    # Lấy số trang, mặc định = 1 nếu không có key 'pages'
    page_range = data_1.get('pages', 1)

    for page in range(1, page_range + 1):
        paged_url = full_url + f"&page={page}"
        r = requests.get(paged_url)
        page_json = r.json()

        if not page_json or len(page_json) < 2 or page_json[1] is None:
            continue

        data = page_json[1]
        df = pd.json_normalize(data)
        all_dfs.append(df)

    if all_dfs:  # chỉ concat nếu có dữ liệu
        final_df = pd.concat(all_dfs, ignore_index=True)
        final_df.to_csv(
            f"/home/anhtt1/Workspace/DE/Project/DE_Project/projects/AlphaVantage/data/{alpha3}_Inflation.csv",
            index=False
        )
        print(f"✅ Đã lưu dữ liệu cho {alpha3}")
    else:
        print(f"⚠ Không có dữ liệu hợp lệ cho {alpha3}")


✅ Đã lưu dữ liệu cho AFG
⚠ Không có dữ liệu cho ALA
✅ Đã lưu dữ liệu cho ALB
✅ Đã lưu dữ liệu cho DZA
✅ Đã lưu dữ liệu cho ASM
✅ Đã lưu dữ liệu cho AND
✅ Đã lưu dữ liệu cho AGO
⚠ Không có dữ liệu cho AIA
⚠ Không có dữ liệu cho ATA
✅ Đã lưu dữ liệu cho ATG
✅ Đã lưu dữ liệu cho ARG
✅ Đã lưu dữ liệu cho ARM
✅ Đã lưu dữ liệu cho ABW
✅ Đã lưu dữ liệu cho AUS
✅ Đã lưu dữ liệu cho AUT
✅ Đã lưu dữ liệu cho AZE
✅ Đã lưu dữ liệu cho BHS
✅ Đã lưu dữ liệu cho BHR
✅ Đã lưu dữ liệu cho BGD
✅ Đã lưu dữ liệu cho BRB
✅ Đã lưu dữ liệu cho BLR
✅ Đã lưu dữ liệu cho BEL
✅ Đã lưu dữ liệu cho BLZ
✅ Đã lưu dữ liệu cho BEN
✅ Đã lưu dữ liệu cho BMU
✅ Đã lưu dữ liệu cho BTN


KeyboardInterrupt: 